<a href="https://colab.research.google.com/github/rizvand/Word-Embedding-BERT/blob/master/BERT_Word_and_Sentence_Embeddings_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 4.2MB/s 
     |████████████████████████████████| 870kB 53.4MB/s 
     |████████████████████████████████| 3.3MB 55.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=f7c9c471910764d34a55530c370c1e04ad02571d6d468c40b3ac9c217e94ab25
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
import torch

# Load Model and Tokenizer

IndoBERT is a state-of-the-art language model for Indonesian based on the BERT model. The pretrained model is trained using a masked language modeling (MLM) objective and next sentence prediction (NSP) objective.

In [3]:
from transformers import BertTokenizer, AutoModel
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1", output_hidden_states = True)

Perbedaan antara tokenizer.encode(text) dengan tokenizer.tokenize(text) adalah tokenizer.encode(text) mengeluarkan output berupa token ids (sudah termasuk token id untuk special token [CLS] di awal kalimat dan [SEP] di akhir kalimat setiap kata). Sedangkan tokenizer.tokenize(text) hanya melakukan tokenisasi dari string -> list of string dari vocabulary yang sudah ada dengan metode WordPiece. Vocabulary memetakan integer ke sebuah kata, untuk mengecek vocabulary dari pre-trained tokenizer dapat digunakan tokenizer.vocab

In [4]:
text = 'Budi bermain bola'

text_tokenized = tokenizer.tokenize(text)
text_encoded = tokenizer.encode(text)
text_decoded = tokenizer.decode(text_encoded)

print('tokenizer.tokenize(text): ', text_tokenized)
print('tokenizer.encode(text): ', text_encoded)
print('tokenizer.decode(text_encoded): ', text_decoded)

tokenizer.tokenize(text):  ['budi', 'bermain', 'bola']
tokenizer.encode(text):  [2, 5103, 1326, 1522, 3]
tokenizer.decode(text_encoded):  [CLS] budi bermain bola [SEP]


Perbedaan antara tokenizer.encode dengan tokenizer.encode_plus terletak pada output yang dihasilkan.

tokenizer.encode_plus menghasilkan sebuah dictionary dengan salah satu keys dari dictionary tersebut adalah 'input_ids' dengan value berupa hasil encoding dari sebuah kalimat tersebut dalam bentuk token ids. Selain hasil encoding, terdapat beberapa tambahan informasi seperti 'token_type_ids' dan 'attention_mask' yang tidak bisa didapatkan dari tokenizer.encode(). Jika tertarik hanya untuk mendapatkan 'input_ids' maka dapat digunakan tokenizer.encode() saja.

Perhatikan bahwa kita ingin melakukan encoding dari sebuah kalimat 'Budi bermain bola', secara intuitif dapat digunakan tiga variasi di bawah. Akan tetapi terdapat perbedaan hasil output untuk ketiga variasi tersebut.

In [5]:
output_1 = tokenizer.encode_plus(['Budi', 'bermain', 'bola'])
output_2 = tokenizer.encode_plus('Budi bermain bola')
output_3 = tokenizer.encode_plus(['Budi bermain bola'])

print('output 1: ', output_1)
print('output 2: ', output_2)
print('output 3: ', output_3)

output 1:  {'input_ids': [2, 1, 1326, 1522, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}
output 2:  {'input_ids': [2, 5103, 1326, 1522, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}
output 3:  {'input_ids': [2, 1, 3], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


Hasil output 1: 'Budi' dianggap sebagai unknown token [UNK] karena keys dalam vocabulary tokenizer ini seluruhnya dalam bentuk lowercase.

Hasil output 3: String 'Budi bermain bola' dianggap sebagai 1 token dan token tersebut tidak terdapat dalam vocabulary tokenizer ini. 

In [6]:
print('decode output 1: ', tokenizer.decode(output_1['input_ids']))
print('decode output 2: ', tokenizer.decode(output_2['input_ids']))
print('decode output 3: ', tokenizer.decode(output_3['input_ids']))

decode output 1:  [CLS] [UNK] bermain bola [SEP]
decode output 2:  [CLS] budi bermain bola [SEP]
decode output 3:  [CLS] [UNK] [SEP]


Misalkan kita memiliki multiple sentences, maka untuk melakukan encoding secara sekaligus dapat digunakan tokenizer.batch_encode_plus(list_of_sentences).

In [7]:
multi_sentences = ['Budi bermain bola.', 'Andi bermain gitar'] 
indexed_tokens = tokenizer.batch_encode_plus(multi_sentences)
indexed_tokens

{'input_ids': [[2, 5103, 1326, 1522, 30470, 3], [2, 8704, 1326, 6891, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

Karena BERT Pytorch menerima input berupa torch.tensor maka dilakukan konversi terlebih dahulu dari list ke torch.tensor.

Sebagai contoh kita akan mengonversi token ids ke dalam torch.tensor untuk kalimat pertama, yaitu 'Budi bermain bola'

In [16]:
tokens_tensor = torch.tensor([indexed_tokens['input_ids'][0]])

Berikutnya merupakan hal yang menarik dari semua ini, yaitu mendapatkan word/sentence vectors menggunakan model BERT. 

Jika support CUDA, maka dapat digunakan konfigurasi berikut, untuk mendapatkan akses gpu di google colab jangan lupa untuk mengganti hardware accelerator menjadi gpu pada tipe runtime

In [51]:
with torch.no_grad():
  output = model(tokens_tensor)

Jika output_hidden_states = True, maka output akan berukuran tiga dimensi, dengan dimensi pertama merupakan output untuk last hidden state dari transformer (1, #tokens, 768), dimensi kedua merupakan pooling dari last hidden state (1, 768), dan dimensi ketiga merupakan semua output dari seluruh hidden state, terdapat total 13 hidden state. Jika output_hidden_states = False, maka output berukuran dua dimensi, tanpa output dari seluruh hidden state.

In [55]:
last_hidden_state = output[0]
pooler_output = output[1]
hidden_state = output[2]

Salah satu teknik untuk mendapatkan word vectors dari setiap token adalah dengan menjumlahkan empat hidden state terakhir dari sebuah model BERT. Sedangkan untuk mendapatkan sentence vectors diperoleh dengan menjumlahkan dua hidden state terakhir

In [76]:
word_vec = (hidden_state[-1][0] + hidden_state[-2][0] + hidden_state[-3][0] + hidden_state[-4][0])

In [77]:
token_vecs = (hidden_state[-2][0] + hidden_state[-1][0])/2

In [83]:
sent_vec = torch.mean(token_vecs, dim = 0)

In [85]:
sent_vec.shape

torch.Size([768])

This notebook is highly inspired by Chris McCormick blog post

Chris McCormick and Nick Ryan. (2019, May 14). BERT Word Embeddings Tutorial. Retrieved from http://www.mccormickml.com

# Test

In [123]:
def sentence_vector(text, model, tokenizer):
  encoded_text = tokenizer.encode_plus(text)
  tokens_tensor = torch.tensor([encoded_text['input_ids']])
  with torch.no_grad():
    output = model(tokens_tensor)
  hidden_states = output[2]

  # using last 2 hidden states averaging strategy
  token_vecs = (hidden_states[-2][0] + hidden_states[-1][0])/2
  sent_vec = torch.mean(token_vecs, dim = 0)
  return sent_vec

In [124]:
from transformers import BertTokenizer, AutoModel
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1", output_hidden_states = True)

In [130]:
list_of_sentence = ['Jokowi sedang berkunjung ke ibukota',
                    'Presiden Indonesia mengunjungi jakarta',
                    'Andi mengunjungi jakarta']

In [131]:
sent_vecs=[]
for sentence in list_of_sentence:
  sent_vec = sentence_vector(sentence, model, tokenizer)
  sent_vecs.append(sent_vec)

In [132]:
from scipy.spatial.distance import cosine

In [136]:
print('vector similarity: ', 1-cosine(sent_vecs[0], sent_vecs[1]))

vector similarity:  0.7629343867301941


# TODO: Build FAISS Index